In [1]:
import psycopg2
import psycopg2.extras
import pandas as pd
import numpy as np
import collections
from datetime import datetime
from scipy import stats
import math 
import pycountry
import re
from geotext import GeoText

In [2]:
conn = psycopg2.connect(dbname='aact',port = 5433)

with conn.cursor(cursor_factory=psycopg2.extras.DictCursor) as cur:
    cur.execute('with sponsors1 as \
                (select *, concat(sponsors.lead_or_collaborator,\':\', sponsors.name) as sponsor_name from sponsors)\
                , grouped_sponsors as \
                (select nct_id, string_agg(sponsor_name, \',\') as sponsor_list from sponsors1 group by nct_id ) \
                ,grouped_interventions as \
                (select nct_id,intervention_type, string_agg(name, \',\') as drug_name from interventions where intervention_type =\'Drug\' group by nct_id,intervention_type) \
                ,grouped_condition as \
                (select nct_id, string_agg(name, \',\') as condition_name from conditions group by nct_id) \
                select studies.nct_id,grouped_interventions.drug_name, grouped_condition.condition_name  , studies.is_fda_regulated_drug,grouped_sponsors.sponsor_list \
                from studies, grouped_sponsors, grouped_condition, grouped_interventions \
                where studies.nct_id = grouped_sponsors.nct_id and grouped_condition.nct_id = studies.nct_id and grouped_interventions.nct_id = studies.nct_id ;')
    column_names = [desc[0] for desc in cur.description]
    aact=cur.fetchall()
    cur.execute('select sponsors.name, string_agg(CAST(@sponsors.id as varchar(10)),\',\') as sponsor_study_list \
                from sponsors \
                group by sponsors.name ;')
    column_names2 = [desc[0] for desc in cur.description]
    aact_sponsor=cur.fetchall()
    

conn.close()

In [3]:
aact = pd.DataFrame(data=aact,columns=column_names)
aact = aact.dropna(subset=['drug_name'])
aact_sponsor =  pd.DataFrame(data=aact_sponsor,columns=column_names2)

In [4]:
# # dropping passed values
# aact.drop(['drug_name'],axis=1)
# aact.reset_index()
aact.head()

,nct_id,drug_name,condition_name,is_fda_regulated_drug,sponsor_list
0,NCT00000130,Intravitreal Antibiotics,"Endophthalmitis,Eye Infections",None,lead:National Eye Institute (NEI)
1,NCT00000102,Nifedipine,Congenital Adrenal Hyperplasia,None,lead:National Center for Research Resources (N...
2,NCT00000194,Cycloserine,Opioid-Related Disorders,None,"lead:Yale University,collaborator:National Ins..."
3,NCT00000199,Piracetam,Cocaine-Related Disorders,None,"lead:University of Pennsylvania,collaborator:N..."
4,NCT00000222,Buprenorphine,Opioid-Related Disorders,None,"lead:National Institute on Drug Abuse (NIDA),c..."


In [5]:
def filter_institute(l):
    new_l=[]
    for ele in l:
        if check_if_company(ele):
            new_l.append(l)
    return new_l
def check_if_insititude(s:str):
    if 'institut' in s or 'research' in s or 'university' in s or 'department' in s:
        return False
    else:
        return True
def clear_white_space(l):
    new_l=[]
    for ele in l:
        while (ele!='' and ele[0]==' '):
            ele = ele[1:]
        while (ele!='' and (ele[-1]==' ' or ele[-1]=='.')):
            ele = ele[:-1]
        if ele!='':
            new_l.append(ele)
    return new_l
def check_if_person_title(s):
    titles=['phd','ph.d','phd.','ph.d.','md phd','m.d','md','m.d.','dmsc','d.o','prof','dr.']
    for title in titles:
        if title in s:
            return False
    return True
def clear_some_common_syntax(l):
    words=['ltd.','ltd','llc','llc.','inc','inc.','pllc','co.','co']
    for word in words:
        if word in l:
            l.remove(word)
    return l

def convert_if_city(s:str):
    if s==None:
        return None
    l = re.split(' |\n',s)
    new_l = []
    for word in l:
        new_l.append(word.capitalize())
    new_s = ' '.join(new_l)
    places = GeoText(new_s)
    cities = places.cities
    if cities:
        return cities[0]
    return None

def convert_if_country_2(s:str):
    if s==None:
        return None
    l = re.split(' |\n',s)
    new_l = []
    for word in l:
        new_l.append(word.capitalize())
    new_s = ' '.join(new_l)
    places = GeoText(new_s)
    countries = places.country_mentions
    if countries:
        values_view = countries.keys()
        value_iterator = iter(values_view)
        first_value = next(value_iterator)
        return first_value
    return None
    
def convert_if_country(s:str):
    if s==None:
        return None
    if 'hong kong' in s:
        return 'Hong kong'
    elif 'uk' in s:
        return 'United Kingdom'
    elif 'columbia' in s:
        return 'Columbia'
    elif 'russia' in s:
        return 'Russia'
    l=re.split(' |\n',s)
    for word in l:
        try:
            country = pycountry.countries.lookup(word)
        except LookupError:
            continue
        return country.name
    try:
        country = pycountry.countries.lookup(s)
    except LookupError:
        return None
    return country.name

In [6]:
aact_sponsor['name'] = aact_sponsor['name'].map(lambda s: s.lower())
aact_sponsor = aact_sponsor[aact_sponsor['name'].map(check_if_insititude)]
aact_sponsor = aact_sponsor[aact_sponsor['name'].map(check_if_person_title)]
aact_sponsor['name'] = aact_sponsor['name'].map(lambda s: s.split(','))
aact_sponsor = aact_sponsor[aact_sponsor['name'].map(lambda l: len(l)>1)]
aact_sponsor['name'] = aact_sponsor['name'].map(clear_white_space)
aact_sponsor['name'] = aact_sponsor['name'].map(clear_some_common_syntax)
# Add location
# Filter country
aact_sponsor['location'] = aact_sponsor['name'].map(lambda l: l[-1] if len(l)>1 else None)
aact_sponsor['location'] = aact_sponsor['location'].map(convert_if_country) 
aact_sponsor = aact_sponsor.reset_index()
for _,row in aact_sponsor.iterrows():
    if row['location']!= None:
        del row['name'][-1]
# Filter city/state
aact_sponsor['country'] = aact_sponsor['location']
aact_sponsor['location'] = aact_sponsor['name'].map(lambda l: l[-1] if len(l)>1 else None)
aact_sponsor['location'] = aact_sponsor['location'].map(convert_if_city) 
aact_sponsor = aact_sponsor.reset_index()
for _,row in aact_sponsor.iterrows():
    if row['location']!= None:
        del row['name'][-1]
# Add more city/country
aact_sponsor['city'] = aact_sponsor['location']
aact_sponsor['location'] = aact_sponsor['name'].map(lambda l: ' '.join(l) if len(l)>1 else None)
aact_sponsor['location'] = aact_sponsor['location'].map(convert_if_city)
for _,row in aact_sponsor.iterrows():
    if row['location']!= None and row['city'] == None:
        row['city'] = row['location']
aact_sponsor['location'] = aact_sponsor['name'].map(lambda l: ' '.join(l) if len(l)>1 else None)
aact_sponsor['location'] = aact_sponsor['location'].map(convert_if_country_2)       
for i,row in aact_sponsor.iterrows():
    if row['location']!= None and row['country'] == None:
        row['country'] = pycountry.countries.lookup(row['location']).name

aact_sponsor = aact_sponsor.drop(columns=['location'])
aact_sponsor['name_len'] = aact_sponsor['name'].map(lambda l: len(l))
aact_sponsor_1 = aact_sponsor[aact_sponsor['name'].map(lambda l: len(l)>1)]
pd.set_option("display.max_rows", None, "display.max_columns", None)
aact_sponsor_1['name'] = aact_sponsor['name'].map(lambda l: ' '.join(l))
aact_sponsor_1

/var/folders/n1/fpyhdnv943b_ffbq6qwk496h0000gn/T/ipykernel_87004/4098612139.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aact_sponsor_1['name'] = aact_sponsor['name'].map(lambda l: ' '.join(l))


,level_0,index,name,sponsor_study_list,country,city,name_len
1,1,2,krka d.d,1684050,Slovenia,Novo Mesto,2
16,16,103,galderma laboratories l.p,"1683248,1687625,1687919,1688252,1690641,169224...",None,None,2
18,18,111,tibotec therapeutics a division of ortho biote...,"1683914,1685541,1687364,1696514,1702249,170690...",United States,None,3
19,19,124,sanofi pasteur a sanofi company,"1683313,1683536,1683626,1683850,1684360,168437...",None,None,2
21,21,131,arbelaez ana maria,1684108,None,None,2
27,27,213,county of onondaga nys,1683421,None,None,2
31,31,273,hospira now a wholly owned subsidiary of pfizer,"1683561,1687464,1688866,1689104,1689169,169238...",None,None,2
36,36,314,mcneil consumer & specialty pharmaceuticals a ...,"1683525,1755850,1760037,1768148,1768165,176819...",None,None,2
38,38,335,genzyme a sanofi company,"1683474,1683795,1683797,1683975,1684356,168451...",None,None,2
44,44,366,janssen korea korea,"1684001,1684016,1684027,1684029,1684219,168469...",None,None,2


In [7]:
aact_sponsor_1.nlargest(100,'name_len')

,level_0,index,name,sponsor_study_list,country,city,name_len
8667,8667,61910,r antakia a xanthis f georgiades v hudson j as...,1646485,None,None,12
470,470,3590,better you ltd unit 24 shortwood court shortwo...,1603394,None,None,7
3563,3563,25773,universitätsklinikum gießen ukgm gmbh justus-l...,1605915,None,None,6
10336,10336,73757,pagano l aversa f locatelli f nosari a rossi g...,1845908,None,None,6
1257,1257,9005,junta de andalucía consejería de conocimiento ...,1976535,None,None,5
1626,1626,11678,sleepiz ag hornbachstrasse 23 8008 zurich info...,1595492,None,None,5
1809,1809,13075,jain isha b.a massachusetts general hospital h...,1197045,None,None,5
2003,2003,14274,the cell therapy facility the blood bank dept....,1629096,Denmark,None,5
2136,2136,15167,fisher daniel r.r.t massachusetts general hosp...,1197044,None,None,5
4127,4127,29877,vivantes klinikum im friedrichshain klinik für...,1605914,None,Berlin,5


In [8]:
import json 
data = json.load(open('drug-label-0001-of-0010.json'))

fda = pd.json_normalize(data["results"])  
features = [col for col in fda.columns if col.startswith('openfda')]
fda=fda[features]
fda.columns = fda.columns.str.lstrip("openfda")
fda.columns = fda.columns.str.lstrip(".")

In [9]:
data = json.load(open('drug-ndc-0001-of-0001.json'))

fda2 = pd.json_normalize(data["results"]) 
fda2.columns

Index(['product_ndc', 'generic_name', 'labeler_name', 'brand_name',
       'active_ingredients', 'finished', 'packaging',
       'listing_expiration_date', 'marketing_category', 'dosage_form',
       'spl_id', 'product_type', 'route', 'marketing_start_date', 'product_id',
       'application_number', 'brand_name_base', 'openfda.manufacturer_name',
       'openfda.rxcui', 'openfda.spl_set_id', 'openfda.is_original_packager',
       'openfda.upc', 'openfda.nui', 'openfda.pharm_class_cs',
       'openfda.pharm_class_epc', 'openfda.unii', 'pharm_class',
       'openfda.pharm_class_moa', 'marketing_end_date', 'dea_schedule',
       'openfda.pharm_class_pe', 'brand_name_suffix'],
      dtype='object')

In [10]:
# strip bracket
fda['application_number']=fda['application_number'].map(lambda string: np.nan
                        if pd.isnull(string) else str(string)[1:-1] ).astype('string')
fda['brand_name']=fda['brand_name'].map(lambda string: np.nan
                        if pd.isnull(string) else str(string)[1:-1] ).astype('string')
fda['generic_name']=fda['generic_name'].map(lambda string: np.nan
                        if pd.isnull(string) else str(string)[1:-1] ).astype('string')
fda['manufacturer_name']=fda['manufacturer_name'].map(lambda string: np.nan
                        if pd.isnull(string) else str(string)[1:-1] ).astype('string')
fda['product_type']=fda['product_type'].map(lambda string: np.nan
                        if pd.isnull(string) else str(string)[1:-1] ).astype('string')
fda['is_original_packager']=fda['is_original_packager'].map(lambda string: np.nan
                        if pd.isnull(string) 
                        else (True if str(string)[1:-1].lower() =='true' else False) ).astype('bool')




In [11]:
fda= fda.drop(columns=['route','rxcui','spl_id','package_ndc','pharm_class_epc','pharm_class_pe',
                 'unii','upc','original_packager_product_ndc','pharm_class_cs','pharm_class_moa'])

In [12]:
fda.head()

,application_number,brand_name,generic_name,manufacturer_name,product_ndc,product_type,substance_name,spl_set_id,is_original_packager,nui
0,'ANDA204559',"'Polyethylene Glycol 3350, Sodium Chloride, So...","'POLYETHYLENE GLYCOL 3350, SODIUM CHLORIDE, SO...",'Strides Pharma Science Limited',"[64380-768, 64380-769, 64380-770]",'HUMAN PRESCRIPTION DRUG',"[POLYETHYLENE GLYCOL 3350, SODIUM CHLORIDE, SO...",[37c111ec-5b62-42b9-bbb2-3b0c530e34c9],True,"[N0000010288, N0000175811, N0000009871]"
1,'ANDA207709','Hydrocodone Bitartrate and Acetaminophen','HYDROCODONE BITARTRATE AND ACETAMINOPHEN',"'Aurolife Pharma, LLC'","[13107-211, 13107-212, 13107-213]",'HUMAN PRESCRIPTION DRUG',"[HYDROCODONE BITARTRATE, ACETAMINOPHEN]",[392a846e-7325-462d-b19b-8afda44c5cb0],True,NaN
2,'ANDA208963','fluconazole','FLUCONAZOLE',"'NuCare Pharmaceuticals,Inc.'",[68071-2601],'HUMAN PRESCRIPTION DRUG',[FLUCONAZOLE],[d35c476f-44c4-1d5c-e053-2a95a90ae3f9],True,"[N0000175487, M0002083, N0000182140, N00001821..."
3,'part352','Eucerin Oil-Control SPF 50 Sunscreen',"'AVOBENZONE, HOMOSALATE, OCTISALATE, OCTOCRYLENE'",'Beiersdorf Inc',[66800-6000],'HUMAN OTC DRUG',"[AVOBENZONE, HOMOSALATE, OCTISALATE, OCTOCRYLENE]",[cda04484-ba84-2276-e053-2a95a90a41c8],True,NaN
4,'ANDA206137','olmesartan medoxomil / amlodipine besylate / ...,'OLMESARTAN MEDOXOMIL / AMLODIPINE BESYLATE / ...,"'Par Pharmaceutical, Inc.'","[49884-786, 49884-787, 49884-788, 49884-789, 4...",'HUMAN PRESCRIPTION DRUG',"[OLMESARTAN MEDOXOMIL, AMLODIPINE BESYLATE, HY...",[268d12e8-1cfc-4086-b613-8fd94f2d7c4e],True,"[N0000175359, N0000175419, M0471776]"


In [13]:
fda.info()
fda['manufacturer_name'].value_counts()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   application_number    9266 non-null   string
 1   brand_name            10659 non-null  string
 2   generic_name          10659 non-null  string
 3   manufacturer_name     10659 non-null  string
 4   product_ndc           10659 non-null  object
 5   product_type          10659 non-null  string
 6   substance_name        10422 non-null  object
 7   spl_set_id            10659 non-null  object
 8   is_original_packager  20000 non-null  bool  
 9   nui                   2731 non-null   object
dtypes: bool(1), object(4), string(5)
memory usage: 1.4+ MB


'Bryant Ranch Prepack'                               286
'A-S Medication Solutions'                           265
'REMEDYREPACK INC.'                                  190
'NuCare Pharmaceuticals,Inc.'                        171
'Washington Homeopathic Products'                    158
'Boiron'                                             154
'Proficient Rx LP'                                   144
'Uriel Pharmacy Inc.'                                121
'Aphena Pharma Solutions - Tennessee, LLC'            80
'PD-Rx Pharmaceuticals, Inc.'                         76
'CVS Pharmacy'                                        71
'The Procter & Gamble Manufacturing Company'          63
'Preferred Pharmaceuticals Inc.'                      60
'Cardinal Health'                                     56
'Major Pharmaceuticals'                               56
'RedPharm Drug, Inc.'                                 55
'Rite Aid Corporation'                                53
'Strategic Sourcing Services LL

In [14]:
# found = 0
# searched = 0
# for i in range(len(fda)):
#     brand_names = fda.loc[i,'brand_name']
# #     generic_names = fda.loc[i,'generic_name']
#     if pd.isna(brand_names):
#         continue
#     brand_names = brand_names.split(',')
#     for j in range(len(aact)):
#         for name in brand_names:
#             try: 
#                 searched = searched +1
#                 if name and name in aact.loc[j,'drug_name']:
#                     found= found + 1
#                     print(f"{brand_names}: {aact.loc[j,'drug_name']} {aact.loc[j,'is_fda_regulated_drug']}")
#             except TypeError :
#                 pass
#             except KeyError :
#                 pass
# print(searched)
# found

In [15]:
convert_if_country('uk')
pycountry.countries.lookup('columbia').name

LookupError: Could not find a record for 'columbia'

In [ ]:
convert_if_country_2(' '.join(['the city hiv centre',' st petersburg',' russia']))

In [ ]:
pycountry.countries.lookup('')